In [24]:
import json
import pennylane as qml
import pennylane.numpy as np

In [252]:
num_wires = 4
dev = qml.device("default.mixed", wires=num_wires)

@qml.qnode(dev)
def heisenberg_trotter(couplings, p, time, depth):
    """This QNode returns the final state of the spin chain after evolution for a time t, 
    under the Trotter approximation of the exponential of the Heisenberg Hamiltonian.
    
    Args:
        couplings (list(float)): 
            An array of length 4 that contains the coupling constants and the magnetic field 
            strength, in the order [J_x, J_y, J_z, h].
        p (float): The depolarization probability after each CNOT gate.
        depth (int): The Trotterization depth.
        time (float): Time during which the state evolves

    Returns:
        (numpy.tensor): The evolved quantum state.
    """

    # Put your code here #
    def noisy_CNOT(p,wires):
        qml.CNOT(wires)
        qml.DepolarizingChannel(p,wires[-1])
    
    jx, jy, jz, h = couplings    
    dt = time / depth
    
    for _ in range(depth):
        qml.RY(0.,0)

        for i in range(4):
            noisy_CNOT(p,[i,(i+1)%4])
            qml.RX(-2*jx*dt, i)
            noisy_CNOT(p,[i,(i+1)%4])
        
        for i in range(4):
            qml.RX(np.pi/2., i)
            qml.RX(np.pi/2., (i+1)%4)
            noisy_CNOT(p,[i,(i+1)%4])
            qml.RZ(-2*jy*dt,(i+1)%4)
            noisy_CNOT(p,[i,(i+1)%4])
            qml.RX(-np.pi/2., i)
            qml.RX(-np.pi/2., (i+1)%4)
        
        for i in range(4):
            noisy_CNOT(p,[i,(i+1)%4])
            qml.RZ(-2*jz*dt, (i+1)%4)
            noisy_CNOT(p,[i,(i+1)%4])
        
        for i in range(num_wires):
            qml.RX(-2*h*dt, wires=i)   

    return qml.state()

def noisy_IsingZ(theta,p):
    for i in range(4):
        noisy_CNOT(p,[i,(i+1)%4])
        qml.RZ(theta, (i+1)%4)
        noisy_CNOT(p,[i,(i+1)%4])
        
def noisy_IsingY(theta,p):
    for i in range(4):
            qml.RX(np.pi/2., i)
            qml.RX(np.pi/2., (i+1)%4)
            noisy_CNOT(p,[i,(i+1)%4])
            qml.RZ(theta,(i+1)%4)
            noisy_CNOT(p,[i,(i+1)%4])
            qml.RX(-np.pi/2., i)
            qml.RX(-np.pi/2., (i+1)%4)
        
def noisy_IsingX(theta,p):
    for i in range(4):
        noisy_CNOT(p,[i,(i+1)%4])
        qml.RX(theta, i)
        noisy_CNOT(p,[i,(i+1)%4])
        
def noisy_CNOT(p,wires):
    qml.CNOT(wires)
    qml.DepolarizingChannel(p,wires[-1])


def calculate_fidelity(couplings, p, time, depth):
    """This function returns the fidelity between the final states of the noisy and
    noiseless Trotterizations of the Heisenberg models, using only CNOT and rotation gates

    Args:
        couplings (list(float)): 
            A list with the J_x, J_y, J_z and h parameters in the Heisenberg Hamiltonian, as
            defined in the problem statement.
        p (float): The depolarization probability of the depolarization gate that acts on the
                   target qubit of each CNOT gate.
        time (float): The period of time evolution simulated by the Trotterization.
        depth (int): The Trotterization depth.

    Returns:
        (float): Fidelity between final states of the noisy and noiseless Trotterizations
    """
    return qml.math.fidelity(heisenberg_trotter(couplings,0,time, depth),heisenberg_trotter(couplings,p,time,depth))

In [246]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output =calculate_fidelity(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises: 
            ``AssertionError`` if the solution output is incorrect in any way.
            
    """
    def create_hamiltonian(params):

        couplings = [-params[-1]]
        ops = [qml.PauliX(3)]

        for i in range(3):

            couplings = [-params[-1]] + couplings
            ops = [qml.PauliX(i)] + ops        

        for i in range(4):

            couplings = [-params[-2]] + couplings
            ops = [qml.PauliZ(i)@qml.PauliZ((i+1)%4)] + ops
        
        for i in range(4):

            couplings = [-params[-3]] + couplings
            ops = [qml.PauliY(i)@qml.PauliY((i+1)%4)] + ops

        for i in range(4):

            couplings = [-params[0]] + couplings
            ops = [qml.PauliX(i)@qml.PauliX((i+1)%4)] + ops    

        return qml.Hamiltonian(couplings,ops)

    @qml.qnode(dev)
    def evolve(params, time, depth):

        qml.ApproxTimeEvolution(create_hamiltonian(params), time, depth)

        return qml.state()
    
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    
    tape = heisenberg_trotter.qtape
    names = [op.name for op in tape.operations]
    
    random_params = np.random.uniform(low = 0.8, high = 3.0, size = (4,) )
    
    assert qml.math.fidelity(heisenberg_trotter(random_params,0,1,2),evolve(random_params,1,2)) >= 1, "Your circuit does not Trotterize the Heisenberg Model"
    
    assert names.count('ApproxTimeEvolution') == 0, "Your circuit must not use the built-in PennyLane Trotterization"
     
    assert set(names) == {'DepolarizingChannel', 'RX', 'RY', 'RZ', 'CNOT'}, "Your circuit must only use RX, RY, RZ, CNOT, and depolarizing gates (don't use qml.Rot or Paulis)"
    
    assert solution_output >= expected_output-0.005, "Your fidelity is not high enough. You may be using more CNOT gates than needed"

In [253]:
test_cases = [['[[1,2,1,0.3],0.05,2.5,1]', '0.33723981123369573'], ['[[1,3,2,0.3],0.05,2.5,2]', '0.15411351752086694']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[1,2,1,0.3],0.05,2.5,1]'...
[0.96325451 2.42281047 1.70238433 2.37350681]
Correct!
Running test case 1 with input '[[1,3,2,0.3],0.05,2.5,2]'...
[2.47512252 2.2411089  2.16492754 2.89687319]
Correct!


In [147]:
def create_hamiltonian(params):

    couplings = [-params[-1]]
    ops = [qml.PauliX(3)]

    for i in range(3):

        couplings = [-params[-1]] + couplings
        ops = [qml.PauliX(i)] + ops        

    for i in range(4):

        couplings = [-params[-2]] + couplings
        ops = [qml.PauliZ(i)@qml.PauliZ((i+1)%4)] + ops

    for i in range(4):

        couplings = [-params[-3]] + couplings
        ops = [qml.PauliY(i)@qml.PauliY((i+1)%4)] + ops

    for i in range(4):

        couplings = [-params[0]] + couplings
        ops = [qml.PauliX(i)@qml.PauliX((i+1)%4)] + ops    

    return qml.Hamiltonian(couplings,ops)

In [38]:
random_params = np.random.uniform(low = 0.8, high = 3.0, size = (4,) )

ham = create_hamiltonian(random_params)

<Hamiltonian: terms=16, wires=[0, 1, 2, 3]>


In [186]:
random_params

tensor([1.75108561, 1.17078937, 0.96572783, 2.00196101], requires_grad=True)

In [195]:
dev = qml.device('default.qubit',wires=4)
@qml.qnode(dev)
def circuit():
    qml.templates.ApproxTimeEvolution(ham, 1, 2)
    return qml.expval(qml.PauliZ(0))

In [196]:
qml.templates.ApproxTimeEvolution(ham, 1, 2).decomposition()

[PauliRot(-1.7510856140701994, XX, wires=[3, 0]),
 PauliRot(-1.7510856140701994, XX, wires=[2, 3]),
 PauliRot(-1.7510856140701994, XX, wires=[1, 2]),
 PauliRot(-1.7510856140701994, XX, wires=[0, 1]),
 PauliRot(-1.1707893679452939, YY, wires=[3, 0]),
 PauliRot(-1.1707893679452939, YY, wires=[2, 3]),
 PauliRot(-1.1707893679452939, YY, wires=[1, 2]),
 PauliRot(-1.1707893679452939, YY, wires=[0, 1]),
 PauliRot(-0.9657278255989906, ZZ, wires=[3, 0]),
 PauliRot(-0.9657278255989906, ZZ, wires=[2, 3]),
 PauliRot(-0.9657278255989906, ZZ, wires=[1, 2]),
 PauliRot(-0.9657278255989906, ZZ, wires=[0, 1]),
 PauliRot(-2.0019610113697985, X, wires=[2]),
 PauliRot(-2.0019610113697985, X, wires=[1]),
 PauliRot(-2.0019610113697985, X, wires=[0]),
 PauliRot(-2.0019610113697985, X, wires=[3]),
 PauliRot(-1.7510856140701994, XX, wires=[3, 0]),
 PauliRot(-1.7510856140701994, XX, wires=[2, 3]),
 PauliRot(-1.7510856140701994, XX, wires=[1, 2]),
 PauliRot(-1.7510856140701994, XX, wires=[0, 1]),
 PauliRot(-1.170